# **Import Library**

In [9]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

## **Set Data to Predict**

In [18]:
# Use Lung Cancer data 
cancer = {'gender': 'M',
    'age': 53,
    'smoking': 2,
    'yellow_fingers': 2,
    'anxiety': 2,
    'peer_pressure': 2,
    'chronic_disease': 2,
    'fatigue': 1,
    'allergy': 2,
    'wheezing': 1,
    'alcohol_consuming': 2,
    'coughing': 1,
    'shortness_of_breath': 1,
    'swallowing_difficulty': 2,
    'chest_pain': 2}

health = {
    'gender': 'F',
    'age': 59,
    'smoking': 1,
    'yellow_fingers': 1,
    'anxiety': 1,
    'peer_pressure': 2,
    'chronic_disease': 1,
    'fatigue': 2,
    'allergy': 1,
    'wheezing': 2,
    'alcohol_consuming': 1,
    'coughing': 2,
    'shortness_of_breath': 2,
    'swallowing_difficulty': 1,
    'chest_pain': 2
}

# **Initiate Some Function to Preprocessing Data**

In [11]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, "utf-8")]
        ),
    )
    
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

In [12]:
def prepare_json(inputs: dict):
    data = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, int):
            data[keys] = int_feature(values)
        elif isinstance(values, str):
            data[keys] = string_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=data)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [15]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)
    
    endpoint = "https://lung-cancer.up.railway.app/v1/models/lung-cancer-model:predict"
    response = requests.post(endpoint, data=json_data)
    prediction = response.json()

    prediction = response.json()["predictions"][0][0]
    
    if prediction < 0.6:
        return "Healthy Lungs"
    else:
        return "Lungs Cancer"

## **Run the Prediction**

In [19]:
# Predict with lungs cancer data
make_predictions(cancer)

'Lungs Cancer'

In [20]:
# Predict with healthy lungs data
make_predictions(health)

'Healthy Lungs'